## Imports

In [1]:
import torch
import pandas as pd
import ltn
import numpy as np

from data_generation import *
from predicates import *
from axioms import *
from trainer import *
from metrics import compute_metrics
from queries import *

## Dataset de treinamento feito em sala

![](image.png)

Houve uma confusão na equipe durante o exercício em sala para criar o dataset de treino do trabalho:
- Acabamos colocando mais do que 5 formas requeridas (elipse, heágono e losango)
- As cores não ficaram one-hot. Na verdade, isso não era um problema, pois conversamos com o professor durante a tarefa sobre fazer uma configuração RGB binária (0 ou 1 em cada canal). O problema é que o dataset entregue no exercício não havia objetos de cor preta e azul (não ciano). Além disso, a cor branca ficou difícil de visualizar. 

Ainda temos as características de cada objeto, por isso conseguimos compreender os 25 objetos:


| Formato: ID | Tipo | Coordenadas (X,Y) | Tamanho | Rotacao | Cor RGB |
| ----------- | ---- | ----------------- | ------- | ------- | ------- |
|  1 | Quadrado   | ( 0.8399,  0.5064) |  0.0483 |  330.0° | (1,0,1) |
|  2 | Quadrado   | ( 0.8567,  0.3403) |  0.0563 |   47.6° | (1,1,1) |
|  3 | Cone       | ( 0.4397,  0.4068) |  0.0276 |  296.9° | (1,1,1) |
|  4 | Circulo    | ( 0.4370,  0.2973) |  0.0255 |   65.0° | (1,0,0) |
|  5 | Triangulo  | ( 0.2523,  0.7002) |  0.0617 |  113.1° | (1,1,0) |
|  6 | Hexagono   | ( 0.2139,  0.1940) |  0.0367 |   64.7° | (1,1,0) |
|  7 | Losango    | ( 0.7305,  0.3057) |  0.0497 |   93.8° | (1,0,0) |
|  8 | Elipse     | ( 0.8327,  0.7886) |  0.0427 |  276.9° | (1,0,1) |
|  9 | Cilindro   | ( 0.2571,  0.8614) |  0.0396 |  265.9° | (0,1,1) |
| 10 | Circulo    | ( 0.7263,  0.8446) |  0.0268 |   20.9° | (1,1,0) |
| 11 | Triangulo  | ( 0.5949,  0.5001) |  0.0360 |  340.5° | (1,1,1) |
| 12 | Circulo    | ( 0.5785,  0.6765) |  0.0473 |   75.5° | (0,1,0) |
| 13 | Cone       | ( 0.6152,  0.8060) |  0.0360 |  332.1° | (1,0,0) |
| 14 | Triangulo  | ( 0.4930,  0.5430) |  0.0306 |  299.9° | (1,0,0) |
| 15 | Cilindro   | ( 0.4446,  0.7856) |  0.0326 |   65.5° | (1,1,0) |
| 16 | Quadrado   | ( 0.7027,  0.7653) |  0.0284 |   44.3° | (0,1,1) |
| 17 | Cone       | ( 0.1770,  0.5710) |  0.0263 |  165.9° | (0,1,0) |
| 18 | Losango    | ( 0.5993,  0.1377) |  0.0309 |  256.9° | (0,1,1) |
| 19 | Quadrado   | ( 0.4630,  0.6838) |  0.0435 |  151.5° | (1,0,0) |
| 20 | Cone       | ( 0.7353,  0.1987) |  0.0262 |  184.8° | (1,1,0) |
| 21 | Cone       | ( 0.7891,  0.1224) |  0.0434 |   37.2° | (0,1,1) |
| 22 | Hexagono   | ( 0.6330,  0.2222) |  0.0260 |   24.6° | (0,1,0) |
| 23 | Cilindro   | ( 0.7022,  0.5066) |  0.0279 |  100.5° | (0,1,1) |
| 24 | Cilindro   | ( 0.2647,  0.5813) |  0.0332 |  355.4° | (0,1,0) |
| 25 | Triangulo  | ( 0.1745,  0.3731) |  0.0282 |  305.2° | (1,0,1) |

Pedimos perdão pela confusão. Para este trabalhos, vamos fazer as seguintes substituições:
- Formas: 
    - `Hexágono -> Triangulo`
    - `Losango -> Quadrado`
    - `Elipse -> Círculo`
- Cores: 
    - `(1,1,1) -> (0,0,1)`
    - `(0,1,1) -> (0,0,1)`
    - `(1,1,0) -> (0,1,0)`
    - `(1,0,1) -> (1,0,0)`

Além disso, também iremos tirar a coluna rotações que não há necessidade para este trabalho. A tabela final então fica:

| Formato: ID | Tipo | Coordenadas (X,Y) | Tamanho | Cor RGB |
| ----------- | ---- | ----------------- | ------- |  ------- |
|  1 | Quadrado   | ( 0.8399,  0.5064) |  0.0483 | (1,0,0) |
|  2 | Quadrado   | ( 0.8567,  0.3403) |  0.0563 | (0,0,1) |
|  3 | Cone       | ( 0.4397,  0.4068) |  0.0276 | (0,0,1) |
|  4 | Circulo    | ( 0.4370,  0.2973) |  0.0255 | (1,0,0) |
|  5 | Triangulo  | ( 0.2523,  0.7002) |  0.0617 | (0,1,0) |
|  6 | Triangulo   | ( 0.2139,  0.1940) |  0.0367 | (0,1,0) |
|  7 | Quadrado    | ( 0.7305,  0.3057) |  0.0497 | (1,0,0) |
|  8 | Circulo     | ( 0.8327,  0.7886) |  0.0427 | (1,0,0) |
|  9 | Cilindro   | ( 0.2571,  0.8614) |  0.0396 | (0,0,1) |
| 10 | Circulo    | ( 0.7263,  0.8446) |  0.0268 | (0,1,0) |
| 11 | Triangulo  | ( 0.5949,  0.5001) |  0.0360 | (0,0,1) |
| 12 | Circulo    | ( 0.5785,  0.6765) |  0.0473 | (0,1,0) |
| 13 | Cone       | ( 0.6152,  0.8060) |  0.0360 | (1,0,0) |
| 14 | Triangulo  | ( 0.4930,  0.5430) |  0.0306 | (1,0,0) |
| 15 | Cilindro   | ( 0.4446,  0.7856) |  0.0326 | (0,1,0) |
| 16 | Quadrado   | ( 0.7027,  0.7653) |  0.0284 | (0,0,1) |
| 17 | Cone       | ( 0.1770,  0.5710) |  0.0263 | (0,1,0) |
| 18 | Quadrado    | ( 0.5993,  0.1377) |  0.0309 | (0,0,1) |
| 19 | Quadrado   | ( 0.4630,  0.6838) |  0.0435 | (1,0,0) |
| 20 | Cone       | ( 0.7353,  0.1987) |  0.0262 | (0,1,0) |
| 21 | Cone       | ( 0.7891,  0.1224) |  0.0434 | (0,0,1) |
| 22 | Triangulo   | ( 0.6330,  0.2222) |  0.0260 | (0,1,0) |
| 23 | Cilindro   | ( 0.7022,  0.5066) |  0.0279 | (0,0,1) |
| 24 | Cilindro   | ( 0.2647,  0.5813) |  0.0332 | (0,1,0) |
| 25 | Triangulo  | ( 0.1745,  0.3731) |  0.0282 | (1,0,0) |

Uma última coisa que faremos no código é normalizar os tamanhos para que o maior tenha tamanho 1.0 e o menor tenha 0.0, pois os tamanhos gerados ficaram muito estranhos.

In [2]:
objetos = [
    # [x, y, R, G, B, Circ, Quad, Cil, Cone, Tri, Tam]
    [0.8399, 0.5064, 1, 0, 0, 0, 1, 0, 0, 0, 0.0483], # ID 1
    [0.8567, 0.3403, 0, 0, 1, 0, 1, 0, 0, 0, 0.0563], # ID 2
    [0.4397, 0.4068, 0, 0, 1, 0, 0, 0, 1, 0, 0.0276], # ID 3
    [0.4370, 0.2973, 1, 0, 0, 1, 0, 0, 0, 0, 0.0255], # ID 4
    [0.2523, 0.7002, 0, 1, 0, 0, 0, 0, 0, 1, 0.0617], # ID 5
    [0.2139, 0.1940, 0, 1, 0, 0, 0, 0, 0, 1, 0.0367], # ID 6
    [0.7305, 0.3057, 1, 0, 0, 0, 1, 0, 0, 0, 0.0497], # ID 7
    [0.8327, 0.7886, 1, 0, 0, 1, 0, 0, 0, 0, 0.0427], # ID 8 
    [0.2571, 0.8614, 0, 0, 1, 0, 0, 1, 0, 0, 0.0396], # ID 9
    [0.7263, 0.8446, 0, 1, 0, 1, 0, 0, 0, 0, 0.0268], # ID 10
    [0.5949, 0.5001, 0, 0, 1, 0, 0, 0, 0, 1, 0.0360], # ID 11
    [0.5785, 0.6765, 0, 1, 0, 1, 0, 0, 0, 0, 0.0473], # ID 12
    [0.6152, 0.8060, 1, 0, 0, 0, 0, 0, 1, 0, 0.0360], # ID 13
    [0.4930, 0.5430, 1, 0, 0, 0, 0, 0, 0, 1, 0.0306], # ID 14
    [0.4446, 0.7856, 0, 1, 0, 0, 0, 1, 0, 0, 0.0326], # ID 15
    [0.7027, 0.7653, 0, 0, 1, 0, 1, 0, 0, 0, 0.0284], # ID 16
    [0.1770, 0.5710, 0, 1, 0, 0, 0, 0, 1, 0, 0.0263], # ID 17
    [0.5993, 0.1377, 0, 0, 1, 0, 1, 0, 0, 0, 0.0309], # ID 18
    [0.4630, 0.6838, 1, 0, 0, 0, 1, 0, 0, 0, 0.0435], # ID 19
    [0.7353, 0.1987, 0, 1, 0, 0, 0, 0, 1, 0, 0.0262], # ID 20
    [0.7891, 0.1224, 0, 0, 1, 0, 0, 0, 1, 0, 0.0434], # ID 21
    [0.6330, 0.2222, 0, 1, 0, 0, 0, 0, 0, 1, 0.0260], # ID 22
    [0.7022, 0.5066, 0, 0, 1, 0, 0, 1, 0, 0, 0.0279], # ID 23
    [0.2647, 0.5813, 0, 1, 0, 0, 0, 1, 0, 0, 0.0332], # ID 24
    [0.1745, 0.3731, 1, 0, 0, 0, 0, 0, 0, 1, 0.0282]  # ID 25
]

objetos_label = [
    {'posicao': (0.8399, 0.5064), 'cor': 'Vermelho', 'forma': 'Quadrado',  'tamanho': 0.0483},
    {'posicao': (0.8567, 0.3403), 'cor': 'Azul',     'forma': 'Quadrado',  'tamanho': 0.0563},
    {'posicao': (0.4397, 0.4068), 'cor': 'Azul',     'forma': 'Cone',      'tamanho': 0.0276},
    {'posicao': (0.4370, 0.2973), 'cor': 'Vermelho', 'forma': 'Círculo',   'tamanho': 0.0255},
    {'posicao': (0.2523, 0.7002), 'cor': 'Verde',    'forma': 'Triângulo', 'tamanho': 0.0617},
    {'posicao': (0.2139, 0.1940), 'cor': 'Verde',    'forma': 'Triângulo', 'tamanho': 0.0367},
    {'posicao': (0.7305, 0.3057), 'cor': 'Vermelho', 'forma': 'Quadrado',  'tamanho': 0.0497},
    {'posicao': (0.8327, 0.7886), 'cor': 'Vermelho', 'forma': 'Círculo',   'tamanho': 0.0427},
    {'posicao': (0.2571, 0.8614), 'cor': 'Azul',     'forma': 'Cilindro',  'tamanho': 0.0396},
    {'posicao': (0.7263, 0.8446), 'cor': 'Verde',    'forma': 'Círculo',   'tamanho': 0.0268},
    {'posicao': (0.5949, 0.5001), 'cor': 'Azul',     'forma': 'Triângulo', 'tamanho': 0.0360},
    {'posicao': (0.5785, 0.6765), 'cor': 'Verde',    'forma': 'Círculo',   'tamanho': 0.0473},
    {'posicao': (0.6152, 0.8060), 'cor': 'Vermelho', 'forma': 'Cone',      'tamanho': 0.0360},
    {'posicao': (0.4930, 0.5430), 'cor': 'Vermelho', 'forma': 'Triângulo', 'tamanho': 0.0306},
    {'posicao': (0.4446, 0.7856), 'cor': 'Verde',    'forma': 'Cilindro',  'tamanho': 0.0326},
    {'posicao': (0.7027, 0.7653), 'cor': 'Azul',     'forma': 'Quadrado',  'tamanho': 0.0284},
    {'posicao': (0.1770, 0.5710), 'cor': 'Verde',    'forma': 'Cone',      'tamanho': 0.0263},
    {'posicao': (0.5993, 0.1377), 'cor': 'Azul',     'forma': 'Quadrado',  'tamanho': 0.0309},
    {'posicao': (0.4630, 0.6838), 'cor': 'Vermelho', 'forma': 'Quadrado',  'tamanho': 0.0435},
    {'posicao': (0.7353, 0.1987), 'cor': 'Verde',    'forma': 'Cone',      'tamanho': 0.0262},
    {'posicao': (0.7891, 0.1224), 'cor': 'Azul',     'forma': 'Cone',      'tamanho': 0.0434},
    {'posicao': (0.6330, 0.2222), 'cor': 'Verde',    'forma': 'Triângulo', 'tamanho': 0.0260},
    {'posicao': (0.7022, 0.5066), 'cor': 'Azul',     'forma': 'Cilindro',  'tamanho': 0.0279},
    {'posicao': (0.2647, 0.5813), 'cor': 'Verde',    'forma': 'Cilindro',  'tamanho': 0.0332},
    {'posicao': (0.1745, 0.3731), 'cor': 'Vermelho', 'forma': 'Triângulo', 'tamanho': 0.0282}
]

In [3]:
def normalizar_tamanhos(objetos):
    max = objetos[0][-1]
    min = objetos[0][-1]

    for objeto in objetos[1:]:
        if objeto[-1] > max:
            max = objeto[-1]
        if objeto[-1] < min:
            min = objeto[-1]
    
    for objeto in objetos:
        objeto[-1] = (objeto[-1] - min) / (max - min)

    return objetos

In [4]:
objetos = normalizar_tamanhos(objetos)

In [5]:
# Função que retorna um texto legível das labels
def texto_objetos_label(objeto_label):
    if objeto_label['tamanho'] <= 0.5:
        tamanho = 'Pequeno'
    else:
        tamanho = 'Grande'

    return f"{objeto_label['forma']} {tamanho} {objeto_label['cor']} na posição {objeto_label['posicao']}"

In [6]:
# Transforma os objetos em objeto tensor
objects = torch.tensor(objetos, dtype=torch.float32)

print(f"Shape do tensor: {objects.shape}")
print(f"Tensor do primeiro objeto: {objects[0]}")
print(f"Descrição do objeto: {texto_objetos_label(objetos_label[0])}")

Shape do tensor: torch.Size([25, 11])
Tensor do primeiro objeto: tensor([0.8399, 0.5064, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.6298])
Descrição do objeto: Quadrado Pequeno Vermelho na posição (0.8399, 0.5064)


## Treinamento

In [7]:
print("Treinando nosso LTN com o dataset criado manualmente")

print(f"Objetos: {objects.shape}")

sat = train_ltn(create_knowledge_base, objects, epochs=100, lr=0.05, verbose=True)
print(f"satAgg final = {sat:.4f}")

Treinando nosso LTN com o dataset criado manualmente
Objetos: torch.Size([25, 11])
Epoch 0000 | satAgg = 0.4930
Epoch 0005 | satAgg = 0.5949
Epoch 0010 | satAgg = 0.6500
Epoch 0015 | satAgg = 0.6927
Epoch 0020 | satAgg = 0.7352
Epoch 0025 | satAgg = 0.7683
Epoch 0030 | satAgg = 0.7868
Epoch 0035 | satAgg = 0.7969
Epoch 0040 | satAgg = 0.8093
Epoch 0045 | satAgg = 0.8169
Epoch 0050 | satAgg = 0.8265
Epoch 0055 | satAgg = 0.8358
Epoch 0060 | satAgg = 0.8421
Epoch 0065 | satAgg = 0.8473
Epoch 0070 | satAgg = 0.8563
Epoch 0075 | satAgg = 0.8653
Epoch 0080 | satAgg = 0.8716
Epoch 0085 | satAgg = 0.8774
Epoch 0090 | satAgg = 0.8826
Epoch 0095 | satAgg = 0.8875
satAgg final = 0.8905


## Testando com 5 gerações de base de teste aleatórias com 25 amostras cada

In [8]:
all_results = []
for run in range(5):
    seed = run * 20 + 1 #(cálculo da nossa cabeça pra fazer as seeds)

    # 1) Gerar objetos (reprodutível)
    objects = generate_objects(n=25, seed=seed)  # tensor (N,11)
    torch.manual_seed(seed)

    sat_final = sat_agg(*create_knowledge_base(objects))

    # 3) Recalcular axiomas (agrupados) e SATs por grupo
    axi_tax = create_axioms_taxonomia_e_formas(objects)
    axi_spatial = create_axioms_raciocinio_espacial(objects)
    axi_vertical = create_axioms_raciocinio_vertical(objects)
    axi_proximity_restriction = [ax_proximity_restriction(objects)]

    sat_tax = sat_agg(*axi_tax)
    sat_spatial = sat_agg(*axi_spatial)
    sat_vertical = sat_agg(*axi_vertical)
    sat_comp = sat_agg(*axi_proximity_restriction)

    # 4) Queries (LTN value + Ground truth)
    q1_val = q_composite_filtering(objects).value.item()
    q1_gt = float(gt_task4_filtragem_composta(objects))
    q2_val = q_deduction_position(objects).value.item()
    q2_gt = float(gt_task4_deducao_posicao(objects))

    # 5) Predicados — métricas (unários e binários)
    preds_unary = {
        'isCircle': eval_unary_predicate(isCircle, gt_is_circle, objects),
        'isSquare': eval_unary_predicate(isSquare, gt_is_square, objects),
        'isCylinder': eval_unary_predicate(isCylinder, gt_is_cylinder, objects),
        'isCone': eval_unary_predicate(isCone, gt_is_cone, objects),
        'isTriangle': eval_unary_predicate(isTriangle, gt_is_triangle, objects),
        'isSmall': eval_unary_predicate(isSmall, gt_is_small, objects),
        'isBig': eval_unary_predicate(isBig, gt_is_big, objects),
        'isRed': eval_unary_predicate(isRed, gt_is_red, objects),
        'isGreen': eval_unary_predicate(isGreen, gt_is_green, objects),
        'isBlue': eval_unary_predicate(isBlue, gt_is_blue, objects),
    }

    preds_binary = {
        'leftOf': eval_binary_predicate(leftOf, gt_left_of, objects),
        'rightOf': eval_binary_predicate(rightOf, gt_right_of, objects),
        'closeTo': eval_binary_predicate(closeTo, gt_close_to, objects),
        'above': eval_binary_predicate(above, gt_above, objects),
        'below': eval_binary_predicate(below, gt_below, objects),
    }

    # 6) Guardar resultados
    run_result = {
        'run': run,
        'seed': seed,
        'sat_final': sat_final.item(),
        'sat_tax': sat_tax.item(),
        'sat_spatial': sat_spatial.item(),
        'sat_vertical': sat_vertical.item(),
        'sat_comp': sat_comp.item(),
        'q1_val': q1_val, 'q1_gt': q1_gt,
        'q2_val': q2_val, 'q2_gt': q2_gt,
        'preds_unary': preds_unary,
        'preds_binary': preds_binary
    }
    all_results.append(run_result)

    print(f"Execução {run+1} (seed = {seed}):")
    print(f"\tsat_final={sat_final:.4f}")
    print(f"\tTaxonomia e Formas: sat = {sat_tax:.3f}")
    print(f"\tRaciocínio espacial: sat = {sat_spatial:.3f}")
    print(f"\tRaciocínio vertical: sat = {sat_vertical:.3f}")
    print(f"\tq1 (Filtragem Composta) = {q1_val:.3f}/{q1_gt}")
    print(f"\tq2 (Dedução de Posição Absoluta) = {q2_val:.3f}/{q2_gt}")

    accuracy, precision, recall, f1 = [], [], [], []
    for predicado in preds_unary:
        accuracy = preds_unary[predicado][0]
        precision = preds_unary[predicado][1]
        recall = preds_unary[predicado][2]
        f1 = preds_unary[predicado][3]
    print(f"\tPredicados unários: \n\t\taccuracy = {np.mean(accuracy)} | precision = {np.mean(precision)} | recall = {np.mean(recall)} | f1 = {np.mean(f1)}")

    accuracy, precision, recall, f1 = [], [], [], []
    for predicado in preds_binary:
        accuracy = preds_binary[predicado][0]
        precision = preds_binary[predicado][1]
        recall = preds_binary[predicado][2]
        f1 = preds_binary[predicado][3]
    print(f"\tPredicados binários: \n\t\taccuracy = {np.mean(accuracy)} | precision = {np.mean(precision)} | recall = {np.mean(recall)} | f1 = {np.mean(f1)}")

    

Execução 1 (seed = 1):
	sat_final=0.7114
	Taxonomia e Formas: sat = 0.978
	Raciocínio espacial: sat = 0.661
	Raciocínio vertical: sat = 0.895
	q1 (Filtragem Composta) = 0.002/1.0
	q2 (Dedução de Posição Absoluta) = 0.015/1.0
	Predicados unários: 
		accuracy = 1.0 | precision = 0.999999999090909 | recall = 0.999999999090909 | f1 = 0.999999994090909
	Predicados binários: 
		accuracy = 0.92 | precision = 0.9199999999693333 | recall = 0.9199999999693333 | f1 = 0.9199999949693335
Execução 2 (seed = 21):
	sat_final=0.7475
	Taxonomia e Formas: sat = 0.859
	Raciocínio espacial: sat = 0.639
	Raciocínio vertical: sat = 0.941
	q1 (Filtragem Composta) = 0.002/1.0
	q2 (Dedução de Posição Absoluta) = 0.009/1.0
	Predicados unários: 
		accuracy = 1.0 | precision = 0.9999999988888888 | recall = 0.9999999988888888 | f1 = 0.9999999938888888
	Predicados binários: 
		accuracy = 0.95 | precision = 0.9655172413460166 | recall = 0.9333333333022222 | f1 = 0.9491525373421432
Execução 3 (seed = 41):
	sat_final=0

Algumas conclusões sobre o trabalho:
- Conseguimos deixar que o LTN conseguisse ir bem nas tarefas de Taxonomia e Formas e Raciocínio Vertical. O raciocínio espacial teve alguma dificuldade que não o deixa ser satisfatível

- As consultas da tarefa 4 de raciocínio composto infelizmente não foram bem. Uma hora ou outra ele acerta quando o valor ground truth da query é 0, mas é praticamente como se estivesse "chutando" 0 sempre, então é normal que acerte quando for 0. Não temos certeza de como poderíamos melhorá-lo, mas as queries erraram.

- As métricas para os predicados unários (isCircle, isRed, isBig...) e binários (leftOf, rightOf, closeTo...) tiveram boas métricas na acurácia, precisão, recall e f1